In [1]:
import bittensor as bt

bt.logging.set_debug(True)
tensor = bt.subtensor("archive")

start_block_number = 2978959
blocks_to_search = 1000

total_subnets = 34
subnets_range = range(1, total_subnets + 1)

/Users/haritowa/Development/nut-a/nodes/bittensor/copycat/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 345 with name 'Yooldo Verse Mainnet' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(
/Users/haritowa/Development/nut-a/nodes/bittensor/copycat/.venv/lib/python3.10/site-packages/eth_utils/network.py:44: UserWarning: Network 12611 with name 'Astar zkEVM' does not have a valid ChainId. eth-typing should be updated with the latest networks.
  warnings.warn(


2024-05-17 23:38:53.789 |       INFO       | bittensor:loggingmachine.py:247 | Debug enabled.
2024-05-17 23:38:54.410 |       INFO       | bittensor:loggingmachine.py:281 |  - Connected to archive network and wss://archive.chain.opentensor.ai:443/. - 


# Collect current axon version

In [2]:
class AxonInfo:
    def __init__(self, hotkey: str, version: int):
        self.hotkey = hotkey
        self.version = version

    def __repr__(self):
        return f"AxonInfo(hotkey={self.hotkey}, version={self.version})"

def get_axon_infos_for_subnet(net_id: int) -> list[AxonInfo]:
    metagraph = tensor.metagraph(net_id, block=start_block_number)

    uids_list = [id for id in metagraph.uids.tolist() if metagraph.validator_permit[id]]
    neurons = [tensor.neuron_for_uid(uid, net_id, block=start_block_number) for uid in uids_list]
    return [AxonInfo(neuron.hotkey, neuron.prometheus_info.version) for neuron in neurons]

def get_axon_infos_for_all_subnets() -> list[list[AxonInfo]]:
    return [get_axon_infos_for_subnet(net_id) for net_id in subnets_range]

axon_infos = get_axon_infos_for_all_subnets()
axon_infos

[[AxonInfo(hotkey=5HEo565WAy4Dbq3Sv271SAi7syBSofyfhhwRNjFNSM2gP9M2, version=0),
  AxonInfo(hotkey=5HK5tp6t2S59DywmHRWPBVJeJ86T61KjurYqeooqj8sREpeN, version=0),
  AxonInfo(hotkey=5F4tQyWrhfGVcNhoqeiNsR6KjD4wMZ2kfhLj4oHYuyHbZAc3, version=0),
  AxonInfo(hotkey=5Hddm3iBFD2GLT5ik7LZnT3XJUnRnN8PoeCFgGQgawUVKNm8, version=0),
  AxonInfo(hotkey=5HYDdYq7sYG22uWyUPZvWqNyTMJn2p6QHVhR8poWB5jhTwQx, version=0),
  AxonInfo(hotkey=5Fq5VTm5bwxgWk3xs9qF7HZY233XB7VjNUYKQP6j2htjwDvx, version=0),
  AxonInfo(hotkey=5EhvL1FVkQPpMjZX4MAADcW42i3xPSF1KiCpuaxTYVr28sux, version=370),
  AxonInfo(hotkey=5CJM2uppAgJCcCAzsneh12rUvYQBkARUrez9B9V2JN7r4e9y, version=0),
  AxonInfo(hotkey=5GWbXamwep4qYHLB5wse19YVLxoioV9mHiUzJcogyizMLN48, version=0),
  AxonInfo(hotkey=5FCnt7CpJ8AYjX3KSxU5Ujt2ju3k3JdT229DCLvKEgGdXBC3, version=0),
  AxonInfo(hotkey=5Dfah4nBEVqAF9qbDzaTWE4pmvAxgfRxqRPZyHtfkf3YJucq, version=0),
  AxonInfo(hotkey=5DkNUTRkn6Yjq7kjckNi21nR3zFjsVQMXvFBHpGzsmA71HGb, version=0),
  AxonInfo(hotkey=5Dw1XUne1EpNkN3rxBnh

# Collect latest submitted version

In [3]:
from dataclasses import dataclass
from scalecodec import GenericExtrinsic

@dataclass
class Extrinsic:
    txHash: str
    signer: str
    success: bool
    subnet: int
    version: int

substrate = tensor.substrate

def get_set_weight_extrinsics(block_number: int) -> list[Extrinsic]:
    block = substrate.get_block(block_number=block_number)
    events = substrate.get_events(block['header']['hash'])

    failed_extrinsics = []

    for event in events:
        event_data = event.value['event']
        if event_data['event_id'] == 'ExtrinsicFailed':
            failed_extrinsics.append(event.value['extrinsic_idx'])

    extrinsics = block['extrinsics']
    result = []

    for index, extrinsic in enumerate(extrinsics):
        extrinsic = extrinsic.value
        
        call = extrinsic['call']
        if call['call_function'] == 'set_weights':
            extrinsic_hash = extrinsic['extrinsic_hash']
            signer = extrinsic['address']
            success = failed_extrinsics.count(index) == 0
            
            call_args = call['call_args']
            subnet = call_args[0]['value']
            version = call_args[-1]['value']
            result.append(Extrinsic(extrinsic_hash, signer, success, subnet, version))

    return result
    
def get_set_weight_extrinsics_all(starting_block_number: int, blocks_to_search: int) -> list[Extrinsic]:
    result = []
    for i in range(blocks_to_search):
        block_number = starting_block_number - i
        try:
            result += get_set_weight_extrinsics(block_number)
        except Exception as e:
            print(f"Error at block {block_number}: {e}")
    return result

all_extrinsics = get_set_weight_extrinsics_all(start_block_number, blocks_to_search)
all_extrinsics

[Extrinsic(txHash='0xc4dba502904b4268799b905362d813957f7cd866e6bc3af8a58ade0c4f0a86fe', signer='5Dd8gaRNdhm1YP7G1hcB1N842ecAUQmbLjCRLqH5ycaTGrWv', success=False, subnet=18, version=0),
 Extrinsic(txHash='0x12038e1b72d4ddf33d5bc2a77ec55117ccdb6d686cb1776f605e8e540ab6ad6d', signer='5Hddm3iBFD2GLT5ik7LZnT3XJUnRnN8PoeCFgGQgawUVKNm8', success=False, subnet=1, version=20200),
 Extrinsic(txHash='0x39ef72880f8c776b0cf0978d92dc2c11540b48b56eec9f0460d495d7c6caef2d', signer='5HK5tp6t2S59DywmHRWPBVJeJ86T61KjurYqeooqj8sREpeN', success=True, subnet=8, version=200),
 Extrinsic(txHash='0x1c02603040a2096585f0fec8c4de386e914c7c69d22067c460d36b023e4d7d3a', signer='5DnXm2tBGAD57ySJv5SfpTfLcsQbSKKp6xZKFWABw3cYUgqg', success=True, subnet=7, version=223),
 Extrinsic(txHash='0x44ab1685ef5e6d3c98cf4e411c08c3bbdc26d683ea756c06ad41dd7457fbb5bb', signer='5DvTpiniW9s3APmHRYn8FroUWyfnLtrsid5Mtn5EwMXHN2ed', success=False, subnet=24, version=0),
 Extrinsic(txHash='0x1caa161e2a5477528b82cacd6e2657194f46ff4a17960327700

In [4]:
# Subnet ID -> Hotkey -> First non faild extrinsic
aggregated_extrinsics = {}

for extrinsic in all_extrinsics:
    if extrinsic.success:
        subnet = extrinsic.subnet
        hotkey = extrinsic.signer
        success = extrinsic.success

        if not success:
            continue
        
        if subnet not in aggregated_extrinsics:
            aggregated_extrinsics[subnet] = {}

        if hotkey not in aggregated_extrinsics[subnet]:
            aggregated_extrinsics[subnet][hotkey] = extrinsic

# Merge data

In [12]:
# Subnet ID -> Hotkey -> Object with min_version axon_version and extrinsics_version
result = {}

for index, subnet in enumerate(subnets_range):
    try:
        extrinsics = aggregated_extrinsics[subnet]
        axons = axon_infos[index]

        result[subnet] = {}
        
        for axon in axons:
            hotkey = axon.hotkey
            version = axon.version

            if hotkey in extrinsics:
                extrinsic = extrinsics[hotkey]
                extrinsic_version = extrinsic.version
            else:
                extrinsic_version = None

            result[subnet][hotkey] = {
                "axon_version": version,
                "subm_version": extrinsic_version,
                "transaction": extrinsic.txHash if extrinsic_version is not None else None
            }
    except Exception as e:
        print(f"Error at subnet {subnet}: {e}")

result

Error at subnet 25: 25


{1: {'5HEo565WAy4Dbq3Sv271SAi7syBSofyfhhwRNjFNSM2gP9M2': {'axon_version': 0,
   'subm_version': 20200,
   'transaction': '0x53b409429fb60e27d42777e8bb9f8e28ae990b5dae07ff4e278023208cb5a86f'},
  '5HK5tp6t2S59DywmHRWPBVJeJ86T61KjurYqeooqj8sREpeN': {'axon_version': 0,
   'subm_version': 20200,
   'transaction': '0xf5f888d7685c47780cd89ecf6f2b48e92b93951bd7ead3c35d087d9f30f88ab9'},
  '5F4tQyWrhfGVcNhoqeiNsR6KjD4wMZ2kfhLj4oHYuyHbZAc3': {'axon_version': 0,
   'subm_version': 20200,
   'transaction': '0x99369c1d2355d9990dbcfa3f16d8756d4c627250ac6b1ee8fda6b76d270392b9'},
  '5Hddm3iBFD2GLT5ik7LZnT3XJUnRnN8PoeCFgGQgawUVKNm8': {'axon_version': 0,
   'subm_version': 20200,
   'transaction': '0xd4588bd7de8c7dba885a66178b31245415b516fc360d969ab3f1cd81c6ab181e'},
  '5HYDdYq7sYG22uWyUPZvWqNyTMJn2p6QHVhR8poWB5jhTwQx': {'axon_version': 0,
   'subm_version': None,
   'transaction': None},
  '5Fq5VTm5bwxgWk3xs9qF7HZY233XB7VjNUYKQP6j2htjwDvx': {'axon_version': 0,
   'subm_version': None,
   'transaction': 

In [15]:
# Convert to json
import json

json_result = json.dumps(result, indent=4)
print(json_result)

{
    "1": {
        "5HEo565WAy4Dbq3Sv271SAi7syBSofyfhhwRNjFNSM2gP9M2": {
            "axon_version": 0,
            "subm_version": 20200,
            "transaction": "0x53b409429fb60e27d42777e8bb9f8e28ae990b5dae07ff4e278023208cb5a86f"
        },
        "5HK5tp6t2S59DywmHRWPBVJeJ86T61KjurYqeooqj8sREpeN": {
            "axon_version": 0,
            "subm_version": 20200,
            "transaction": "0xf5f888d7685c47780cd89ecf6f2b48e92b93951bd7ead3c35d087d9f30f88ab9"
        },
        "5F4tQyWrhfGVcNhoqeiNsR6KjD4wMZ2kfhLj4oHYuyHbZAc3": {
            "axon_version": 0,
            "subm_version": 20200,
            "transaction": "0x99369c1d2355d9990dbcfa3f16d8756d4c627250ac6b1ee8fda6b76d270392b9"
        },
        "5Hddm3iBFD2GLT5ik7LZnT3XJUnRnN8PoeCFgGQgawUVKNm8": {
            "axon_version": 0,
            "subm_version": 20200,
            "transaction": "0xd4588bd7de8c7dba885a66178b31245415b516fc360d969ab3f1cd81c6ab181e"
        },
        "5HYDdYq7sYG22uWyUPZvWqNyTMJn2p6QHVhR8p